In [1]:
!pip install roboflow ultralytics

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 1.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.27.0 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.6 which is incompatible.


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="vw7zsJifR7NC94OxTV4e")
project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Car-3 in yolov11:: 100%|██████████| 59606/59606 [05:28<00:00, 181.58it/s]


In [3]:
import os
import yaml
import shutil
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/mahmoud/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
base_dir = 'C:\working\Self-Driving-Car-3'
yaml_path = os.path.join('C:\working\Self-Driving-Car-3', 'data.yaml')
export_images = os.path.join(base_dir, 'export', 'images')
export_labels = os.path.join(base_dir, 'export', 'labels')
valid_images = os.path.join(base_dir, 'valid', 'images')
valid_labels = os.path.join(base_dir, 'valid', 'labels')
results_dir = 'C:\working/results/autonomous_detection'

os.makedirs(valid_images, exist_ok=True)
os.makedirs(valid_labels, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

is_valid_empty = not os.path.exists(valid_images) or len(os.listdir(valid_images)) == 0


In [16]:


if is_valid_empty and os.path.exists(export_images) and len(os.listdir(export_images)) > 0:
    print("Creating validation set by splitting export data...")
    
    # Get list of image files
    image_files = [f for f in os.listdir(export_images) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Split into training and validation sets
    if len(image_files) >= 5:  # Ensure enough images for splitting
        _, valid_files = train_test_split(image_files, test_size=0.2, random_state=42)
        
        # Copy validation files
        for img_file in valid_files:
            # Copy image
            shutil.copy2(
                os.path.join(export_images, img_file),
                os.path.join(valid_images, img_file)
            )
            
            # Copy corresponding label file
            label_file = os.path.splitext(img_file)[0] + '.txt'
            if os.path.exists(os.path.join(export_labels, label_file)):
                shutil.copy2(
                    os.path.join(export_labels, label_file),
                    os.path.join(valid_labels, label_file)
                )
        
        print(f"Created validation set with {len(valid_files)} images.")
    else:
        print("Too few images for splitting. Using training folder for validation as well.")


In [17]:
with open(yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

data_yaml['train'] = export_images 

if os.path.exists(valid_images) and len(os.listdir(valid_images)) > 0:
    data_yaml['val'] = valid_images
else:
    print("No images in validation folder. Using training folder for validation as well.")
    data_yaml['val'] = export_images

# Write the updated data.yaml file
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print("\nUpdated data.yaml configuration:")
print(f"Training: {data_yaml['train']}")
print(f"Validation: {data_yaml['val']}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\working\\Self-Driving-Car-3/data.yaml'

In [ ]:
print("\nExploring dataset:")
if os.path.exists(export_images):
    num_images = len([f for f in os.listdir(export_images) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Total training images: {num_images}")
    
    # Display class distribution if possible
    if 'names' in data_yaml:
        print(f"Classes in dataset: {data_yaml['names']}")


In [14]:
import os
import wandb

os.environ["WANDB_API_KEY"] = "05b9b63a10722e4a10d8cc7663f9c1e847c766ec" 
os.environ["WANDB_PROJECT"] = "autonomous_vehicle_detection"
os.environ["WANDB_ENTITY"] = "amrdabour24"  

In [ ]:
# Add this to your training setup code
import wandb

# Initialize wandb with more config details
wandb.init(
    project="autonomous_vehicle_detection",
    name="yolov8_self_driving",
    config={
        "model": "yolov8n",
        "epochs": 5,
        "batch_size": 16,
        "image_size": 640,
        "optimizer": "Adam",
        "learning_rate": 0.001
    }
)

# During training, create custom logs for visualization
# Add this after each validation step
def log_metrics_with_custom_viz(results):
    # Log standard metrics
    wandb.log({
        "box_loss": results.box_loss,
        "cls_loss": results.cls_loss,
        "dfl_loss": results.dfl_loss,
        "mAP50": results.metrics.map50,
        "mAP50-95": results.metrics.map,
        "precision": results.metrics.precision,
        "recall": results.metrics.recall
    })
    
    # Create custom visualization
    wandb.log({
        "loss_comparison": wandb.plot.line_series(
            xs=list(range(len(results.box_loss_history))),
            ys=[
                results.box_loss_history,
                results.cls_loss_history,
                results.dfl_loss_history
            ],
            keys=["Box Loss", "Classification Loss", "DFL Loss"],
            title="Training Losses",
            xname="Epochs"
        )
    })
    
    # Add performance visualization
    wandb.log({
        "detection_performance": wandb.plot.line_series(
            xs=list(range(len(results.map50_history))),
            ys=[
                results.map50_history,
                results.map_history,
                results.precision_history,
                results.recall_history
            ],
            keys=["mAP50", "mAP50-95", "Precision", "Recall"],
            title="Detection Performance",
            xname="Epochs"
        )
    })

In [ ]:
print("\nStarting model training...")

model = YOLO('yolov8n.pt')  #  'yolov8s.pt', 'yolov8m.pt', or 'yolov8l.pt'

# Initialize wandb run
wandb.init(project="autonomous_vehicle_detection", 
           name="yolov8_self_driving",
           config={
               "model": "yolov8n",
               "epochs": 5,
               "batch_size": 16,
               "image_size": 640,
               "optimizer": "Adam",
               "learning_rate": 0.001
           })

training_params = {
    'data': yaml_path,
    'epochs': 5,           # Number of training epochs
    'patience': 10,         # Early stopping patience
    'batch': 16,            # Batch size
    'imgsz': 640,           # Image size
    'device': 0 if torch.cuda.is_available() else 'cpu',  # Use GPU if available
    'save': True,           # Save model checkpoints
    'project': '/kaggle/working/results',
    'name': 'autonomous_detection',
    'pretrained': True,     # Use pre-trained weights
    'optimizer': 'Adam',    # Optimizer type
    'lr0': 0.001,           # Initial learning rate
    'weight_decay': 0.0005, # Weight decay for regularization
    'cos_lr': True,         # Use cosine learning rate scheduler
    'label_smoothing': 0.1, # Label smoothing
    'augment': True,        # Use data augmentation
}

# YOLOv8 will automatically use wandb since it's initialized
# results = model.train(**training_params)

# Finish wandb run
# wandb.finish()

In [ ]:
# Add this code before your model.train() call
import wandb

# Initialize wandb manually (if not already done)
if wandb.run is None:
    wandb.init(project="autonomous_vehicle_detection", name="yolov8_self_driving")

# Create a custom callback function to log metrics after each epoch
def log_metrics_callback(trainer):
    # Log training losses
    try:
        wandb.log({
            "box_loss": trainer.metrics.get("box_loss", 0),
            "cls_loss": trainer.metrics.get("cls_loss", 0),
            "dfl_loss": trainer.metrics.get("dfl_loss", 0),
            
            # Validation metrics
            "map50": trainer.metrics.get("metrics/mAP50", 0),
            "map": trainer.metrics.get("metrics/mAP50-95", 0),
            "precision": trainer.metrics.get("precision", 0),
            "recall": trainer.metrics.get("recall", 0),
            
            "epoch": trainer.epoch
        })
    except Exception as e:
        print(f"Error logging to wandb: {e}")

# Register the callback with the model
model.add_callback("on_fit_epoch_end", log_metrics_callback)

# Then run training as before
results = model.train(**training_params)

In [ ]:
evaluation = model.val(data=yaml_path)
print(f"Resultados de validación:")
print(f"mAP@0.5: {evaluation.box.map50:.4f}")
print(f"mAP@0.5:0.95: {evaluation.box.map:.4f}")

if hasattr(evaluation.box.p, "mean"):
    print(f"Precision: {evaluation.box.p.mean():.4f}")
else:
    print(f"Precision: {evaluation.box.p:.4f}")
    
if hasattr(evaluation.box.r, "mean"):
    print(f"Recall: {evaluation.box.r.mean():.4f}")
else:
    print(f"Recall: {evaluation.box.r:.4f}")

In [ ]:
print("\nExporting model for deployment...")
for export_format in ['onnx', 'tflite', 'saved_model']:
    try:
        model.export(format=export_format, imgsz=640)
        print(f"Exported model to {export_format} format successfully.")
    except Exception as e:
        print(f"Failed to export model to {export_format} format: {e}")


In [ ]:
print("\nTesting model on sample images...")
for i in range(400,690,5):
    if os.path.exists(valid_images) and len(os.listdir(valid_images)) > 0:
        # Get a sample image from validation set
        sample_img = os.path.join(valid_images, os.listdir(valid_images)[i])
        
        # Run inference
        results = model(sample_img)
        
        # Plot results
        fig, ax = plt.subplots(figsize=(10, 8))
        res_plotted = results[0].plot()
        ax.imshow(res_plotted)
        ax.axis('off')
        plt.savefig(os.path.join(results_dir, 'detection_example.png'))
        plt.show()
        print("Saved detection example image.")

print("\nTraining complete! Model saved to:", results_dir)
print("You can now use this model for real-time object detection in autonomous vehicles.")